In [ ]:
# Prep
import pandas as pd
import os
from os.path import join

data_dir = ''
ess_dir = join(data_dir, 'ess2014')
eurob_path = join(data_dir, 'eurobarometer-96_dk_subset.csv')

## Sammensætning af data

Man kan adskille mellem to overordnede måder at sammensætte data på:

- Concatentation/appending: Datasæt "klaskes" sammen uden videre
- Merge/joins: Datasæt sammensættes på baggrund af en nøgle (fx personnummer)

I denne notebook ses eksempler på, hvordan man sammensætter data på disse måder med `pandas`.

### Concatenation/appending

Concatenation/appending bruges til at sætte datasæt sammen - typisk uden nogen form for validering. Det kan fx bruges, når man har to datasæt med samme variable, men forskellige observationer.

I `pandas` bruges `pd.concat()` til at sammensætte data på denne måde. Funktionen forventer en liste af dataframes, som skal slås sammen.

*BEMÆRK*: Funktionen tillader, at man kan sammensætte langs begge akser (rækker eller kolonner). Som standard antager funktionen, at det er datasættenes rækker, som skal slås sammen (`axis = 0`).

In [ ]:
# stier til data

ess14_1_path = join(ess_dir, 'ess2014_mainsub_p1.csv')
ess14_2_path = join(ess_dir, 'ess2014_mainsub_p2.csv')

# indlæs datasæt
ess14_1 = pd.read_csv(ess14_1_path)
ess14_2 = pd.read_csv(ess14_2_path)

In [ ]:
ess14_1.head()

In [ ]:
ess14_1.shape

In [ ]:
ess14_2.head()

In [ ]:
ess14_2.shape

In [ ]:
# sammensæt data med pd.concat
ess14_comb = pd.concat([ess14_1, ess14_2])

In [ ]:
ess14_comb.shape

### Merge/joins

Merge/joins bruges til at sammensætte variable fra flere datasæt på baggrund af en eller flere nøglevariable, som optræder i de datasæt, som skal slås sammen. Det kan fx være et personnummer eller andet id-nummer. I visse tilfælde bruges flere nøglevariable, hvis fx hver række unikt identificeres ved kombinationen af flere variable (fx vejnavn og postnummer). 

Der findes forskellige typer af joins, alt efter hvordan data skal slås sammen:
- left/right: behold rækker i et datasæt
- outer: behold alle rækker
- inner: behold rækker, som optræder i begge datasæt

I `pandas` bruges funktionen `pd.merge()` til at joine data. Funktionen forventer to datasæt (et "left" datasæt og et "right" datasæt). Man styrer, hvilken type join det er med argumentet `how`. Nøglevariable sættes med argumentet `on`. Hvis variable har forskellige navne i de to datasæt, kan man angive navnet for hhv. det venstre og højre datasæt (`left_on`/`right_on`). 

*BEMÆRK*: Som standard antager funktionen, at der skal foretages et inner join (behold rækker, som optræder i begge datasæt).

In [ ]:
# sti til data
ess14_trst_path = join(ess_dir, 'ess2014_trstsub.csv')

# indlæs data
ess14_trst = pd.read_csv(ess14_trst_path)

In [ ]:
ess14_trst.head()

In [ ]:
ess14_trst.shape

In [ ]:
# left join

ess14_joined1 = pd.merge(ess14_1, ess14_trst, how = 'left', on = 'idno')

In [ ]:
ess14_joined1.head()

In [ ]:
# right join

ess14_joined2 = pd.merge(ess14_1, ess14_trst, how = 'right', on = 'idno')

In [ ]:
ess14_joined2.head()

## Ændring af dataformat

Datasæt kan have forskellige formater og strukturer. Det kan ofte være nødvendigt at ændre på datas format - enten for at det passer med den metode/funktion/model, som det skal bruges i, eller fordi det skal sættes sammen med andre datasæt.

For datasæt i en tabelstruktur (rækker og kolonner), kan man overordnet adskille mellem to formater:
- wide: én række per observeret enhed (fx person), hvor hver oplysning/variabel har sin egen kolonne
- long: en observeret enhed kan have flere rækker, hvor en variabel indikerer, hvilken oplysning der er tale om for enheden

Man støder ofte på det ene eller andet format i forbindelse med tidsserier, hvor man har gentagne målinger for de samme enheder. I wide-format vil man typisk adskille mellem tidsenheder i kolonnerne for at bevare én række per enhed. I long-format vil man adskille mellem tidsenheder i en variabel, så hver enhed ender med flere rækker.

I `pandas` bruges funktionen `melt()` til at konvertere wide-long og `pivot()` til at konvertere long-wide.

### Wide-long konvertering (`melt()`)

I en wide-long konvertering, laver man kolonnenavne om til en variabel, sådan at man finder en unik værdi i datasættet ud fra en nøglevariabel, den nye kolonnevariabel samt den variabel, som man er interesseret i.

For at foretage konverteringen skal man som mininmum angive følgende: 
- En nøglevariabel: Hvordan findes unikke observationer? (`id_vars`)
- Kolonner, som skal formateres om (`value_vars`)
- Navn på variabel, som skal indeholde kolonnenavne (`var_name`)
- Navn på variabel, som skal indeholde værdierne (`value_name`)

In [ ]:
# sti til data

dream_path = join(data_dir, 'bef_dream_2015_sim.csv')

# indlæs data

dream_df = pd.read_csv(dream_path)

In [ ]:
dream_df.head()

In [ ]:
# columns to reshape

cols_reshape = dream_df.columns[dream_df.columns.str.startswith('br')]

# reshape
dream_long = dream_df.melt(id_vars = 'PNR', value_vars = cols_reshape, var_name = 'month_year', value_name = 'branche')
dream_long = dream_long.sort_values(['PNR', 'month_year'])

In [ ]:
dream_long.dropna().head(10)

### Long-wide konvertering (`pivot()`)

I en long-wide konvertering, laver man værdier i en variabel om til kolonnenavne, som tager sin værdi fra en anden variabel. På den måde får hver enhed i data sin egen række, hvor man adskiller mellem værdierne i kolonnerne.

For at foretage konverteringen skal man som mininmum angive følgende: 
- En nøglevariabel: Hvordan findes unikke observationer? (`index`)
- Navn på variabel, som skal laves til kolonnenavne (`columns`)
- Navn på variabel, som indeholder værdier til de nye kolonner (`values`)

In [ ]:
# reshape back

dream_wide = dream_long.pivot(index = 'PNR', columns = 'month_year', values = 'branche')

In [ ]:
dream_wide.head()